In [1]:
import keras
from keras.preprocessing.image import load_img, img_to_array
import os
from PIL import Image
from collections import defaultdict
import pandas as pd
import numpy as np

Using TensorFlow backend.


## Loading Data

In [3]:
folders = os.listdir()

In [4]:
folders

['.ipynb_checkpoints',
 'Ac',
 'As',
 'Cb',
 'Cc',
 'Ci',
 'Cs',
 'Ct',
 'Cu',
 'Ns',
 'Sc',
 'St',
 'Untitled.ipynb']

In [5]:
folders = folders[1:-1]

In [6]:
images = defaultdict(list)

for folder in folders:
    image_names = os.listdir(folder)
    for image_name in image_names:
        images[folder].append(img_to_array(load_img(folder+'/'+image_name, target_size=(256,256))).flatten())

In [7]:
df = pd.DataFrame(columns=['data', 'label'])
for key in images.keys():
    mini_df = pd.DataFrame(columns=['data', 'label'])
    mini_df['data'] = images[key]
    mini_df['label'] = key
    df = pd.concat([df, mini_df])

In [8]:
df.reset_index(inplace=True)

In [9]:
data = np.vstack(df.data.values)

In [10]:
labels = list(df.label)

In [11]:
data.shape

(2543, 196608)

## Preprocessing Data

In [2]:
from sklearn.preprocessing import StandardScaler

scale_obj = StandardScaler()
centered_data = scale_obj.fit_transform(data) #Using standard scaler to make mean 0 for all the features

from sklearn.decomposition import PCA

pca_obj = PCA()
pca_data = pca_obj.fit_transform(centered_data)

NameError: name 'data' is not defined

In [5]:
from matplotlib import pyplot as plt

plt.clf()
plt.plot(range(1,pca_obj.n_components_+1), np.cumsum(pca_obj.explained_variance_ratio_))
plt.grid()
plt.xlabel('Principal Components')
plt.ylabel('Variance Preserved')
plt.show()

AttributeError: 'PCA' object has no attribute 'n_components_'

<Figure size 432x288 with 0 Axes>

Seems from the above graph 1000 principal components are preserving the entire variance

In [ ]:
pca_data_sliced = pca_data[:,:1000]

In [16]:
pca_data_sliced.shape

(2543, 1000)

In [17]:
from sklearn.utils import shuffle

X, y = shuffle(pca_data_sliced, labels)

In [18]:
from sklearn.preprocessing import LabelEncoder

label_enc_obj = LabelEncoder()
num_labels = label_enc_obj.fit_transform(labels)

## Modelling the Data

In [19]:
from sklearn.model_selection import train_test_split

x_tr, x_te, y_tr, y_te = train_test_split(pca_data_sliced, num_labels, stratify=num_labels, shuffle=True, test_size=0.2)

In [42]:
from sklearn.linear_model import LogisticRegression

lr_model = LogisticRegression().fit(X, y)


C:\Users\00004891\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\00004891\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\00004891\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [45]:
from sklearn.model_selection import GridSearchCV, cross_val_score

scores = cross_val_score(LogisticRegression(), X, y, cv=8)

C:\Users\00004891\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\00004891\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\00004891\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\00004891\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\00004891\AppData\Local\Con

In [54]:
lr_model2 = LogisticRegression().fit(x_tr, y_tr)
lr_model.score(x_te, y_te)

C:\Users\00004891\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\00004891\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\00004891\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:182: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  score = y_true == y_pred


0.0

In [55]:
lr_model2.score(x_te, y_te)

0.17092337917485265

In [57]:
lr_model3 = LogisticRegression(C=0.5).fit(x_tr, y_tr)
lr_model3.score(x_te, y_te)

C:\Users\00004891\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\00004891\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.17485265225933203

In [58]:
lr_model3.score(x_tr, y_tr)

0.9862340216322517

In [59]:
parameters = {'C':[0.8, 0.6, 0.4, 0.2, 0.05]}
grid_search_obj = GridSearchCV(LogisticRegression(), parameters, cv=5)
grid_search_obj.fit(X, y)

C:\Users\00004891\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\00004891\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\00004891\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\00004891\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\00004891\AppData\Local\Con

C:\Users\00004891\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\00004891\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\00004891\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\00004891\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='warn',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='warn',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'C': [0.8, 0.6, 0.4, 0.2, 0.05]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [60]:
pd.DataFrame(grid_search_obj.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,95.455929,5.009350,0.008192,0.008360,0.8,{'C': 0.8},0.167641,0.169922,0.147638,0.175889,0.152778,0.162800,0.010717,5
1,88.605883,5.275420,0.003692,0.000728,0.6,{'C': 0.6},0.165692,0.166016,0.155512,0.183794,0.152778,0.164766,0.010869,4
2,75.065873,2.297579,0.004010,0.000026,0.4,{'C': 0.4},0.169591,0.169922,0.161417,0.179842,0.146825,0.165552,0.010986,3
3,64.391927,1.847217,0.003639,0.000759,0.2,{'C': 0.2},0.169591,0.183594,0.163386,0.179842,0.146825,0.168698,0.013038,2
4,47.953554,1.618258,0.003979,0.000029,0.05,{'C': 0.05},0.175439,0.187500,0.188976,0.183794,0.158730,0.178923,0.011090,1


In [3]:
parameters = {'C':[0.01, 0.005, 0.001, 0.0001]}
grid_search_obj2 = GridSearchCV(LogisticRegression(), parameters, cv=5)
grid_search_obj2.fit(X, y)

NameError: name 'GridSearchCV' is not defined

In [64]:
pd.DataFrame(grid_search_obj2.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,31.373978,0.640321,0.004100,0.000180,0.01,{'C': 0.01},0.185185,0.187500,0.204724,0.203557,0.178571,0.191899,0.010397,4
1,26.997232,0.653623,0.004564,0.001103,0.005,{'C': 0.005},0.189084,0.195312,0.212598,0.201581,0.184524,0.196618,0.009834,3
2,16.417632,0.477985,0.004011,0.000036,0.001,{'C': 0.001},0.214425,0.218750,0.230315,0.219368,0.218254,0.220212,0.005336,2
3,6.729803,0.106445,0.003190,0.001595,0.0001,{'C': 0.0001},0.247563,0.257812,0.271654,0.288538,0.246032,0.262289,0.015962,1


In [65]:
parameters = {'C':[0.0001, 0.00001, 0.000001]}
grid_search_obj2 = GridSearchCV(LogisticRegression(), parameters, cv=5)
grid_search_obj2.fit(X, y)

C:\Users\00004891\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\00004891\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\00004891\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\00004891\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='warn',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='warn',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='warn', n_jobs=None, param_grid={'C': [0.0001, 1e-05, 1e-06]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [67]:
pd.DataFrame(grid_search_obj2.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,6.756739,0.031199,0.004953,0.001060,0.0001,{'C': 0.0001},0.247563,0.257812,0.271654,0.288538,0.246032,0.262289,0.015962,3
1,3.354509,0.039511,0.004122,0.002308,1e-05,{'C': 1e-05},0.298246,0.277344,0.297244,0.304348,0.259921,0.287456,0.016439,1
2,1.922349,0.034543,0.003096,0.001578,1e-06,{'C': 1e-06},0.280702,0.253906,0.275591,0.272727,0.259921,0.268580,0.010055,2


In [68]:
grid_search_obj2.score(X, y)

0.4459300039323634

In [20]:
for key in images.keys():
    print(f"No. of {key} images are {len(images[key])}\n")

No. of Ac images are 221

No. of As images are 188

No. of Cb images are 242

No. of Cc images are 268

No. of Ci images are 139

No. of Cs images are 287

No. of Ct images are 200

No. of Cu images are 182

No. of Ns images are 274

No. of Sc images are 340

No. of St images are 202



In [73]:
lr_model_final = LogisticRegression(C=0.00001).fit(x_tr, y_tr)

C:\Users\00004891\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\00004891\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [75]:
lr_model_final.score(x_tr, y_tr)

0.4695181907571288

In [77]:
from sklearn.metrics import classification_report 

print(classification_report(label_enc_obj.inverse_transform(y_te), label_enc_obj.inverse_transform(lr_model_final.predict(x_te))))

              precision    recall  f1-score   support

          Ac       0.09      0.02      0.04        44
          As       0.21      0.26      0.24        38
          Cb       0.31      0.54      0.39        48
          Cc       0.22      0.31      0.26        54
          Ci       0.25      0.04      0.06        28
          Cs       0.25      0.12      0.16        58
          Ct       0.34      0.65      0.45        40
          Cu       0.20      0.08      0.12        36
          Ns       0.09      0.07      0.08        55
          Sc       0.35      0.56      0.43        68
          St       0.20      0.07      0.11        40

    accuracy                           0.27       509
   macro avg       0.23      0.25      0.21       509
weighted avg       0.23      0.27      0.23       509



In [78]:
print(classification_report(label_enc_obj.inverse_transform(y_tr), label_enc_obj.inverse_transform(lr_model_final.predict(x_tr))))

              precision    recall  f1-score   support

          Ac       0.74      0.31      0.43       177
          As       0.38      0.48      0.42       150
          Cb       0.43      0.68      0.53       194
          Cc       0.45      0.65      0.54       214
          Ci       0.65      0.14      0.22       111
          Cs       0.60      0.28      0.38       229
          Ct       0.47      0.73      0.57       160
          Cu       0.60      0.26      0.36       146
          Ns       0.46      0.42      0.43       219
          Sc       0.45      0.71      0.55       272
          St       0.48      0.25      0.33       162

    accuracy                           0.47      2034
   macro avg       0.52      0.44      0.43      2034
weighted avg       0.51      0.47      0.45      2034



In [21]:
from sklearn.svm import SVC

In [23]:
svc_model = SVC().fit(x_tr, y_tr)
print(f'score on training data - {svc_model.score(x_tr, y_tr)}')
print(f'score on testing data - {svc_model.score(x_te, y_te)}')

C:\Users\00004891\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


score on training data - 0.9631268436578171
score on testing data - 0.15324165029469547


In [87]:
parameters = {'C':[0.5, 1.5, 0.9, 0.8, 0.95]}
grid_search_svc = GridSearchCV(SVC(), parameters, cv=5)
grid_search_svc.fit(X, y)

C:\Users\00004891\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\00004891\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\00004891\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning

C:\Users\00004891\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='auto_deprecated', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='warn', n_jobs=None,
             param_grid={'C': [0.5, 1.5, 0.9, 0.8, 0.95]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [90]:
pd.DataFrame(grid_search_svc.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,10.396185,0.474048,1.341002,0.074474,0.5,{'C': 0.5},0.134503,0.132812,0.133858,0.136364,0.136905,0.134880,0.001532,5
1,9.972853,0.101546,1.320215,0.027235,1.5,{'C': 1.5},0.144250,0.150391,0.159449,0.158103,0.146825,0.151789,0.006033,2
2,10.043593,0.201578,1.347202,0.029530,0.9,{'C': 0.9},0.146199,0.148438,0.159449,0.156126,0.148810,0.151789,0.005078,2
3,10.154657,0.340357,1.310395,0.021019,0.8,{'C': 0.8},0.146199,0.148438,0.159449,0.156126,0.148810,0.151789,0.005078,2
4,9.783996,0.125537,1.326456,0.037394,0.95,{'C': 0.95},0.146199,0.150391,0.159449,0.158103,0.148810,0.152576,0.005243,1


In [22]:
from sklearn.model_selection import GridSearchCV, cross_val_score

parameters = {'C':[0.5, 1.5, 0.9, 0.8, 0.95, 0.1], 'kernel':['linear', 'poly', 'sigmoid']}
grid_search_svc2 = GridSearchCV(SVC(gamma='auto', max_iter=1000), parameters, cv=5)
grid_search_svc2.fit(X, y)

C:\Users\00004891\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\svm\base.py:241: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
C:\Users\00004891\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\svm\base.py:241: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
C:\Users\00004891\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\svm\base.py:241: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
C:\Users\00004891\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\svm\base.py:241: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with Sta

C:\Users\00004891\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\svm\base.py:241: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
C:\Users\00004891\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\svm\base.py:241: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
C:\Users\00004891\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\svm\base.py:241: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
C:\Users\00004891\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\svm\base.py:241: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with Sta

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='auto', kernel='rbf', max_iter=1000,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='warn', n_jobs=None,
             param_grid={'C': [0.5, 1.5, 0.9, 0.8, 0.95, 0.1],
                         'kernel': ['linear', 'poly', 'sigmoid']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [25]:
pd.DataFrame(grid_search_svc2.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,5.855840,0.115737,1.232816,0.013939,0.5,linear,"{'C': 0.5, 'kernel': 'linear'}",0.208577,0.203125,0.200787,0.213439,0.228175,0.210775,0.009700,11
1,6.315201,0.019849,1.187006,0.005915,0.5,poly,"{'C': 0.5, 'kernel': 'poly'}",0.251462,0.240234,0.242126,0.233202,0.220238,0.237515,0.010384,2
2,5.478602,0.155303,1.254801,0.016315,0.5,sigmoid,"{'C': 0.5, 'kernel': 'sigmoid'}",0.118908,0.123047,0.122047,0.106719,0.103175,0.114825,0.008208,14
3,5.726807,0.040349,1.232802,0.015861,1.5,linear,"{'C': 1.5, 'kernel': 'linear'}",0.218324,0.195312,0.208661,0.223320,0.236111,0.216280,0.013739,7
4,6.332391,0.031590,1.190416,0.006991,1.5,poly,"{'C': 1.5, 'kernel': 'poly'}",0.251462,0.240234,0.242126,0.231225,0.220238,0.237122,0.010576,5
5,5.261800,0.161565,1.254800,0.015197,1.5,sigmoid,"{'C': 1.5, 'kernel': 'sigmoid'}",0.132554,0.105469,0.118110,0.100791,0.095238,0.110499,0.013405,17
6,5.740800,0.047953,1.228601,0.003665,0.9,linear,"{'C': 0.9, 'kernel': 'linear'}",0.222222,0.203125,0.204724,0.229249,0.202381,0.212348,0.011177,10
7,6.355000,0.102543,1.188813,0.005532,0.9,poly,"{'C': 0.9, 'kernel': 'poly'}",0.251462,0.240234,0.242126,0.233202,0.220238,0.237515,0.010384,2
8,5.450619,0.134557,1.285190,0.046103,0.9,sigmoid,"{'C': 0.9, 'kernel': 'sigmoid'}",0.128655,0.115234,0.110236,0.104743,0.105159,0.112859,0.008815,16
9,5.768191,0.028949,1.230800,0.012897,0.8,linear,"{'C': 0.8, 'kernel': 'linear'}",0.216374,0.197266,0.208661,0.229249,0.224206,0.215100,0.011354,9


In [37]:
svc_model3 = SVC(gamma='auto', max_iter=-1, kernel='poly', C=0.000005).fit(x_tr, y_tr)
print(f'score on training data - {svc_model3.score(x_tr, y_tr)}')
print(f'score on training data - {svc_model3.score(x_te, y_te)}')

score on training data - 0.9414945919370699
score on training data - 0.3005893909626719


### The best score is obtained from an SVM classifier with polynomial kernal of degree 3

In [39]:
from sklearn.metrics import classification_report 
print(classification_report(label_enc_obj.inverse_transform(y_tr), label_enc_obj.inverse_transform(svc_model3.predict(x_tr))))

              precision    recall  f1-score   support

          Ac       0.99      0.92      0.95       177
          As       0.94      0.93      0.93       150
          Cb       0.96      0.99      0.97       194
          Cc       0.88      0.98      0.92       214
          Ci       0.96      0.90      0.93       111
          Cs       0.89      0.95      0.92       229
          Ct       0.99      0.93      0.96       160
          Cu       0.99      0.92      0.95       146
          Ns       0.94      0.94      0.94       219
          Sc       0.96      0.97      0.96       272
          St       0.92      0.88      0.90       162

    accuracy                           0.94      2034
   macro avg       0.95      0.94      0.94      2034
weighted avg       0.94      0.94      0.94      2034



In [40]:
print(classification_report(label_enc_obj.inverse_transform(y_te), label_enc_obj.inverse_transform(svc_model3.predict(x_te))))

              precision    recall  f1-score   support

          Ac       0.11      0.07      0.08        44
          As       0.26      0.24      0.25        38
          Cb       0.49      0.60      0.54        48
          Cc       0.29      0.37      0.32        54
          Ci       0.20      0.18      0.19        28
          Cs       0.28      0.34      0.31        58
          Ct       0.48      0.55      0.51        40
          Cu       0.13      0.11      0.12        36
          Ns       0.28      0.20      0.23        55
          Sc       0.32      0.37      0.34        68
          St       0.17      0.12      0.14        40

    accuracy                           0.30       509
   macro avg       0.27      0.29      0.28       509
weighted avg       0.28      0.30      0.29       509



On test set, the model is performing decently in picking 'cb' and 'ct' categories of clouds.